# Obtener Información desde sitio web

In [1]:
"""

Primero utilizaremos la librería `requests` para obtener información de un sitio web

"""

import requests


In [19]:
"""


Vamos a buscar información de temperatura, ingresamos las coordenadas que deseamos conocer la temperatura



"""

respuesta = requests.get("https://api.open-meteo.com/v1/forecast",
                         params = {"latitude": -4.06496,
                                   "longitude": -78.95529,
                                   "current": "temperature_2m"})

print(respuesta.json())

{'latitude': -4.0, 'longitude': -78.875, 'generationtime_ms': 0.0209808349609375, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 913.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C'}, 'current': {'time': '2024-06-19T03:15', 'interval': 900, 'temperature_2m': 21.6}}


In [20]:
"""
Formateamos la respuesta JSON para que sea más fácil su visualización

"""

from pprint import pprint
pprint(respuesta.json())

{'current': {'interval': 900,
             'temperature_2m': 21.6,
             'time': '2024-06-19T03:15'},
 'current_units': {'interval': 'seconds',
                   'temperature_2m': '°C',
                   'time': 'iso8601'},
 'elevation': 913.0,
 'generationtime_ms': 0.0209808349609375,
 'latitude': -4.0,
 'longitude': -78.875,
 'timezone': 'GMT',
 'timezone_abbreviation': 'GMT',
 'utc_offset_seconds': 0}


## Crear una objeto de aplicación utilizando la librería QUIX

In [21]:
# Crear una nueva aplicación utilizndo Quixstreams

from quixstreams import Application

app = Application(
    broker_address = "localhost:29092",
    loglevel = "DEBUG",
)

In [22]:
# Prepara la medición para ingestarla en Kafka
import json

clima = respuesta.json()

In [24]:
# Obtener un productor que ingrese datos a Kafka

with app.get_producer() as producer:
    producer.produce(
        topic = "quix_demo",
        key   = "Zamora",
        value =  json.dumps(clima)
    )

[2024-06-18 22:29:05,381] [INFO] : Topics required for this application: 
[2024-06-18 22:29:05,383] [DEBUG] : No topics provided for creation...skipping!
[2024-06-18 22:29:05,383] [INFO] : Validating Kafka topics exist and are configured correctly...
[2024-06-18 22:29:05,386] [INFO] : Kafka topics validation complete
[2024-06-18 22:29:05,387] [DEBUG] : Flushing kafka producer
[2024-06-18 22:29:05,390] [DEBUG] : Kafka producer flushed


## Lo convertimos en función

In [25]:
""" Creamos una función que lee los datos en formato JSON """

def leer_temperatura():
    
    respuesta = requests.get("https://api.open-meteo.com/v1/forecast",
                            params = {"latitude": -4.06496,
                                    "longitude": -78.95529,
                                    "current": "temperature_2m"})

    return respuesta.json()

In [30]:
with app.get_producer() as producer:
    temperatura = leer_temperatura()
    producer.produce(
        topic = "quix_demo",
        key = "Zamora",
        value =  json.dumps(temperatura)
    )

[2024-06-18 22:32:02,195] [INFO] : Topics required for this application: 
[2024-06-18 22:32:02,195] [DEBUG] : No topics provided for creation...skipping!
[2024-06-18 22:32:02,196] [INFO] : Validating Kafka topics exist and are configured correctly...
[2024-06-18 22:32:02,199] [INFO] : Kafka topics validation complete
[2024-06-18 22:32:03,003] [DEBUG] : Flushing kafka producer
[2024-06-18 22:32:03,007] [DEBUG] : Kafka producer flushed
